In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
from scipy import constants
import sympy as sp

%matplotlib inline

mpl.rcParams['savefig.dpi'] = 150
mpl.rcParams['figure.dpi'] = 150
mpl.rcParams['font.size'] = 8
mpl.rcParams['font.family'] = 'Arial'

# Model description

## Basic idea

TBD

## Quantum capacitance

We use a very simple model here, where we calculate the quantum capacitance from the tunneling rate between two quantum dots. It is given by

$$
    C_q = \frac{\alpha^2 e^2}{t}, 
$$

where $\alpha$ is the lever arm of the sensing gate on the dot we measure on, and $t$ is the tunneling rate to the other dot. $t$ can of course also be a co-tunneling rate, the only important thing is that we consider a coherent charge tunneling rate here.


## Reflected signal

### High-Q regime

Since we deal with Qs that are fairly high (even in the simple case they seem higher than a simple estimation of a series LC directly coupled to the tx-line suggests), we use an approximation for the reflection coefficient that doesn't require detailed circuit analysis (See Clerk et al., RMP 2010):

$$
    \Gamma = \frac{\omega - \omega_r + i\omega_r/(4Q)}{\omega-\omega_r-i\omega_r/(4Q)}
$$

$\omega$ is the drive frequency, $\omega_r$ is the resonance frequency, and $Q$ is the Q-factor of the oscillator, which is given by $Q = \omega_r / \kappa$, where $\kappa$ is the **energy** decay rate.

The signature of the quantum capacitance is then the phase shift between the reflection off the resonator with quantum capacitance present or not, i.e., between reflection using resonator frequencies $\omega_{r,0}=(LC)^{-1/2}$ and $\omega_{r,q} = (L(C+C_q))^{-1/2}$.


## Relation to readout signal of an MZM qubit

The readout of the quantum capacitance due to coherent tunnel coupling already gives us a good estimate for a MZM qubit readout. We can understand this as follows. The model for the qubit readout is that we have an interferometric coupling between two dots in the form of 

$$
    \omega_{DD} = \left( \epsilon^2 + (t_0 + \sigma_z t_1)^2) \right)^{1/2} \approx t(1 + \sigma_z) = \pm t/2,
$$

where $t_{0,1}$ are the couplings through the two interferometer arms, and $\sigma_z$ is the Pauli operator of the MZM qubit. Ideally we have $\epsilon=0$ and $t_0 = t_1 \equiv t$ during readout (see RHS). From this we can see that the maximal readout signal in reflection between $\sigma_z = \pm1$ is equivalent to the difference between zero and full quantum capacitance value.


### Amplitude of the readout drive

The readout scheme is based on the value of the interdot coupling beeing a function of the MZM parity operator. Importantly, during readout we probe the difference in phase response between two DD charge ground state configurations. We do not wish to excite the DD system during this readout, since the short lifetime of the excited states will lead to decoherence of the DD system (which will have backaction onto the qubit).

While we have ideas on how that could be modeled (see below), for now we use empiric numbers. In first test experiments a power of about -110 dBm at the sample gave decent results. For now we simply fix this power.

We then assume that all power is reflected and measured. 


### Noise

We assume we have only thermal noise here. Then the number of thermal photons that are added to signal is given by the Bose-Einstein distribution, 

$$
    n_{th} = \left( \exp(\frac{\hbar \omega}{kT_N}) - 1 \right)^{-1},
$$

where $T_N$ is the noise temperature of the signal chain.




### SNR and qubit readout fidelity

If the two signals we are measuring result from a phase shift of $\theta$, then after integration we have a separation (in units of photon number) of the two signals of $\Delta = \kappa \bar{n} T_\text{m} \sin \left(\frac{\theta}{2} \right)$, where $T_m$ is the measurement time, and $\bar{n}$ the steady state number of photons in the resonator.

SNR is then given as $\Delta/n_{th}$. This can also be mapped directly onto qubit fidelity, of course (TBD).



# Open


### Hi-Q vs Lo-Q description

* the low-Q model is based on a simple circuit but disagrees with experiments (which give higher Q than what we'd expect)
* the high-Q model is based on only Q, but we don't have a model (i.e., adding quantum capacitance is added to an 'effective capacitance'.

Can we find a better model that takes into account Q, but also ensures we work with $C_q$ correctly?


### Limits on the drive amplitudes

Our basic idea is that drive amplitude must be limited by the RF voltage applied to the gate, which could cause excitation of the charge state of the DD system, leading to undesired decoherence. Some basic ideas on how to model that are given below. These are not yet incorporated into the quantitative model.

#### Direct driving

If we're not far off resonance (i.e., in the limit of small $n_g$ for the RF drive) the drive can induce direct transitions. The Hamiltonian of the DD system in our model is (without drive) 

$$
    H_{DD} = \begin{pmatrix}
        -\epsilon/2 & t/2\\
        t/2 & \epsilon/2
    \end{pmatrix},
$$

i.e., a charge qubit formed by the DD. Close to resonance the quantization axis is given by the tunnel coupling, with DD eigenstates $|L\rangle \pm |R\rangle$. The drive modulates the detuning, and the Hamiltonian in this frame is then (assuming lever arm of unity, i.e., worst case)

$$
    H_{DD} = t \sigma_z^{DD} / 2 + eV_{RF} \cos(\omega t) \sigma_x^{DD} / 2\hbar. 
$$

From this we can estimate the probability with which the drive maximally populates the excited state as

$$
    P_{exc} \sim \frac{(eV_{RF}/\hbar)^2}{(eV_{RF}/\hbar)^2 + (t-\omega)^2}.
$$


#### Adiabaticity condition

Another way to look at it is by adiabiticity: The RF voltage ramp needs to be slow enough so we remain in the GS at all time while modulating the gate. The probability for exciting the system is given by the Landau-Zener relation,

$$
    P_{exc} = 1 - \exp\left( - \dot{\epsilon} / 2 t^2 \right),
$$

where 

$$
    \dot{\epsilon} \equiv \frac{d\epsilon}{dt} = \frac{d}{dt} V_{RF} \cos(\omega t)
$$

which leads us to the required condition

$$
    e V_{RF} \omega \ll 2 t^2.
$$

However: this discussion assumes that each voltage ramp starts and ends far off resonance. The voltages we consider here correspond typically to pretty small $n_g$, though, so that is likely not a good picture.

In [ ]:
twopi = np.pi * 2

In [ ]:
def volt2omega(V):
    return constants.e * V / (constants.hbar)


def volt2nbar(V, omega, C):
    return C * (np.abs(V))**2 / (constants.hbar * omega)


def dBm2pwr(dBm):
    return 1e-3 * 10**(dBm/10)
    

def pwr2dBm(pwr):
    return 10*np.log10(pwr/1e-3)


def pwr2photonflux(pwr, omega):
    return pwr / (constants.hbar * omega)


def dBm2photonflux(dBm, omega):
    return pwr2photonflux(dBm2pwr(dBm), omega)


def impedance(L, C, omega):
    """
    Compute impedance for series LC circuit.
    """
    Z = 1j*omega*L + 1/(1j*omega*C)
    return Z


def reflection(Z, Z_0):
    """
    Compute the voltage reflection coefficient given load impedance and characteristic impedance.
    """
    Gamma = (Z - Z_0)/(Z + Z_0)
    return Gamma


def phase_offset_loQ(L, Cp, Cq):
    omega_r0 = 1/np.sqrt(L*Cp)
    omega_rq = 1/np.sqrt(L*(Cp+Cq))
    Z1 = impedance(L, Cp, omega_r0)
    Z2 = impedance(L, Cp+Cq, omega_r0)
    Gamma1 = reflection(Z1, 50)
    Gamma2 = reflection(Z2, 50)
    phase1 = np.angle(Gamma1, deg=False)
    phase2 = np.angle(Gamma2, deg=False)
    return phase1 - phase2


def reflection_hiQ(omega, omega_r, Q):
    """
    Compute the voltage reflection coefficient for the load being a high-Q resontor.
    Inputs:
        * omega : drive frequency
        * omega_r : resonance frequency of the oscillator
        * Q : Q-factor of the oscillator
    """
    Gamma=(omega-omega_r+1j*omega_r/(4*Q))/(omega-omega_r-1j*omega_r/(4*Q))
    return Gamma


def phase_offset_hiQ(L, Cp, Cq, Q):
    """
    Compute the phase in reflection that is due to quantum capacitance.
    For Q >= 50, we use the high-Q approximation for the reflection coefficient,
    otherwise we use the standard formula or Voltage reflection and assume a simple series LC resonator.
    """
    omega_r0 = 1/np.sqrt(L*Cp)
    omega_rq = 1/np.sqrt(L*(Cp+Cq))
    Gamma1 = reflection_hiQ(omega_r0,omega_r0,Q)
    Gamma2 = reflection_hiQ(omega_r0,omega_rq,Q)
    phase1 = np.angle(Gamma1, deg=False)
    phase2 = np.angle(Gamma2, deg=False)
    return phase1 - phase2


def Cq(alpha, t):
    """
    Compute the quantum capacitance given lever arm alpha and tunnel coupling t.
    """
    return alpha**2 * constants.e**2 / (constants.hbar * t)


# def DD_excitation_probability(omega, V_RF, t):
#     """
#     Compute the excited state probability when driving with V_RF on a DD system near resonance.
#     Inputs:
#         * omega : drive frequency
#         * V_RF : drive amplitude in volts
#         * t : tunnel coupling
#     """
#     frabi = volt2omega(V_RF)
#     return frabi**2 / (frabi**2 + (t-omega)**2)


# def max_V(t, omega, p_max=0.01):
#     """
#     Compute the maximal drive voltage amplitude that results in at most p_max excitation.
#     """
#     return ((p_max * (t-omega)**2)/ ((constants.e/constants.hbar)**2 * (1.-p_max)))**(0.5)


def n_thermal(T_N, omega):
    return (np.exp(constants.hbar * omega / (constants.k * T_N)) - 1)**(-1)

def SNR(photonflux, phase, omega, T_m, T_N):
    noise = n_thermal(T_N, omega)
    signal = photonflux * T_m * np.sin(phase/2.)
    return signal/noise

In [ ]:
pwr2dBm(1000 * constants.k * 3.0 * twopi * 1e6)

In [ ]:
n_thermal(0.022, twopi*1.4e9)

In [ ]:
dBm2photonflux(-40-40-20-10, twopi * 1.4e9)

In [ ]:
SNR(11000, 0.5 * np.pi / 180., twopi * 1.4e9, 100, 6)

In [ ]:
alpha = 0.8
t = twopi * 20e9
Cp = 0.3e-12
Q = 50
T_N = 3.0
G = 1000




omega_vals = twopi * np.linspace(0.5, 3, 6) * 1e9
L_vals = (omega_vals**2 * Cp)**(-1)

Cq_vals = Cq(alpha, t)
phase_vals = phase_offset_hiQ(L_vals, Cp, Cq_vals, Q)
phase_vals_deg = phase_vals * 180 / np.pi
phase_vals_deg[phase_vals_deg < 0] += 360.

nout_vals = dBm2photonflux(-110, omega_vals)

print(nout_vals * 1e-6)

# snr = SNR(max_n_vals, phase_vals, omega_vals, Q, 1e-6, T_N, G)
# n_th = n_thermal(T_N, omega_vals)

# print(max_n_vals * omega_vals/Q * 1e-6)

# fig, ax = plt.subplots(1, 1, figsize=(3,2))
# ax.plot(omega_vals / twopi, phase_vals_deg)

# fig, ax = plt.subplots(1, 1, figsize=(3,2))
# ax.plot(omega_vals / twopi, max_V_vals * 1e6)

# fig, ax = plt.subplots(1, 1, figsize=(3,2))
# ax.plot(omega_vals / twopi, max_n_vals)

# fig, ax = plt.subplots(1, 1, figsize=(3,2))
# ax.plot(omega_vals / twopi, n_th)

# fig, ax = plt.subplots(1, 1, figsize=(3,2))
# ax.plot(omega_vals / twopi, snr)

In [ ]:
volt2nbar(8e-6, twopi*0.5e9, 0.3e-12)

In [ ]:
kappa = twopi * 0.5e9 / 50.
2 * np.arcsin(120./(300000. * 1.4)) * 180/np.pi

In [ ]:
dBm2photonflux(-127, twopi * 193414e9)

In [ ]:
dBm2pwr(0)